In [21]:
# Units
lb = lbs = 1
gallon = gallons = 1
ounce = ounces = 1

In [22]:
# Constants
hlt_deadspace = 2.25*gallons
mash_deadspace = 0.25*gallons
hot_side_tubing_deadspace = 0.25*gallons
cold_side_tubing_deadspace = 0.25*gallons
boil_deadspace = 2.25*gallons

In [23]:
# Formulas
mash_thickness = lambda lbs_grains: lbs_grains*(0.375*(gallons/lb))
grain_absorption = lambda lbs_grains: lbs_grains*(0.125*(gallons/lb))
hop_absorption = lambda ounces_hops: ounces_hops*(0.15*(gallons/ounce))
boil_off = lambda boil_hours: (1.5*gallons)/boil_hours

In [24]:
def how_much_water_do_I_need(lbs_grain=0, ounces_hops=0, boil_hours=0, final_expected_volume=5.5):
    dead_liquid = hlt_deadspace + hot_side_tubing_deadspace + cold_side_tubing_deadspace + boil_deadspace
    lost_to_grains = grain_absorption(lbs_grain)
    lost_to_hops = hop_absorption(ounces_hops)
    evaporation = boil_off(boil_hours)
    start_volume = dead_liquid + lost_to_grains + lost_to_hops + evaporation + final_expected_volume
    strike_volume = mash_thickness(lbs_grain) + mash_deadspace + hot_side_tubing_deadspace
    pre_boil_volume = final_expected_volume + evaporation + lost_to_hops
    sparge_volume = pre_boil_volume + lost_to_grains + cold_side_tubing_deadspace - strike_volume
    return {
        "start_volume": start_volume,
        "strike_volume": strike_volume,
        "sparge_volume": sparge_volume,
        "pre-boil_volume": pre_boil_volume,
        "fermenter_volume": final_expected_volume
    }

In [25]:
how_much_water_do_I_need(lbs_grain=20, ounces_hops=16, boil_hours=1, final_expected_volume=5.5)

{'start_volume': 16.9,
 'strike_volume': 8.0,
 'sparge_volume': 4.15,
 'pre-boil_volume': 9.4,
 'fermenter_volume': 5.5}